In [2]:
from model import *
import pymongo

format_data = {"name": "abc"}
format_instance = Format(**format_data)

site_data = {"name": "Abc"}
site_instance = Site(**site_data)

tournament_data = {
    "name": "abc",
    "buyin": 1,
    "guarantee": 2,
    "site": site_instance,
    "formats": [format_instance],
}
tournament_instance = Tournament(**tournament_data)

sessionStats_data = {"winnings": 10, "position": 10, "tournamnets": tournament_instance}
sessionStats_instance = SessionStats(**sessionStats_data)

session_data = {"date": datetime.now(), "length": 4, "stats": sessionStats_instance}
session = Session(**session_data)

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["pokerDb"]
mycol = mydb["sessions"]

x = mycol.insert_one(session)


In [4]:
import random
from DataFaker import DataFaker


def convert_to_mongo_documents(faker: DataFaker) -> List[dict]:
    mongo_documents = []
    tournament_instances = []  # List to store tournament instances

    # Convert Sites
    for site_data in faker.si_sites_data:
        site_instance = Site(name=site_data[0])
        #mongo_documents.append(site_instance)

    # Convert Formats
    for format_data in faker.f_formats_data:
        format_instance = Format(name=format_data[0])
        #mongo_documents.append(format_instance)

    # Convert Tournaments
    for tournament_index, tournament_data in enumerate(faker.t_tournaments_data):
        site_instance = Site(name=tournament_data[3])
        format_instances = [
            Format(name=format_name) for format_name in faker.tournament_formats_mapping[tournament_index + 1]
        ]

        tournament_instance = Tournament(
            name=tournament_data[0],
            buyin=tournament_data[1],
            guarantee=tournament_data[2],
            site=site_instance,
            formats=format_instances
        )
        tournament_instances.append(tournament_instance)  # Add to the list
        #mongo_documents.append(tournament_instance)

    # Convert Sessions
    for session_data in faker.s_sessions_data:
        # Convert datetime.date to datetime.datetime
        session_date = datetime.combine(session_data[0], datetime.min.time())
        
        session_stats_data = {
            "winnings": random.randint(0, 10000),
            "position": random.randint(1, 500),
            "tournament": [tournament_instance for tournament_instance in tournament_instances]
        }

        session_instance = Session(
            date=session_date,
            length=session_data[1],
            stats=[session_stats_data]
        )
        mongo_documents.append(session_instance)

    return mongo_documents

# Usage
faker = DataFaker()
mongo_documents = convert_to_mongo_documents(faker)

print("datda")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["pokerDb"]

# Insert into MongoDB
collection = mydb["sessions"]
collection.insert_many(mongo_documents)

print("MongoDB data insertion completed.")

datda
MongoDB data insertion completed.
